In [1]:
import re
from nipype import Node
from nipype.interfaces.io import DataFinder

In [2]:
# First get the folder names for the masks and tracks
seedMaskFinder = Node(DataFinder(match_regex = '.*seed.*mask.*\.nii\.gz'), name = 'seedMaskFinder')
targetMaskFinder = Node(DataFinder(match_regex = '.*target*.mask.*\.nii\.gz'), name = 'seedMaskFinder')
trackFolderFinder = Node(DataFinder(match_regex = '/*track/*', unpack_single = True), name = 'trackFolderFinder')

In [100]:
#tracking_dir = '/Users/srothmei/Desktop/charite/toronto/FR_20120903/tractography/'
tracking_dir = '/Users/srothmei/Desktop/charite/toronto/AJ_20140516_1600/mrtrix_68/'
seedMaskFinder.inputs.root_paths = tracking_dir
seedmasks = seedMaskFinder.run()
targetMaskFinder.inputs.root_paths = tracking_dir
targetmasks = targetMaskFinder.run()
trackFolderFinder.inputs.root_paths = tracking_dir
trackFolder = trackFolderFinder.run().outputs.out_paths

# Build subset
subSet = seedmasks.outputs.out_paths[0:10]
subSet = dict(zip(subSet, range(len(subSet))))
#print subSet

for i in subSet.keys():
    # Get the mask id i.e. the last chain of min. 4 digits in the current string
    maskID = re.findall("\d{4,999}", i)[-1][:4]
    # Now find the corresponding mask in the set of possible target masks
    matching = [s for s in targetmasks.outputs.out_paths if maskID in s]
    # Insert into dict
    subSet[i] = matching[0]

print subSet


INFO:workflow:Executing node seedMaskFinder in dir: /var/folders/tg/2g576vk174s3m9znw1sgjpbm0000gn/T/tmp8wL_xg/seedMaskFinder
INFO:workflow:Executing node seedMaskFinder in dir: /var/folders/tg/2g576vk174s3m9znw1sgjpbm0000gn/T/tmpvc16Dx/seedMaskFinder
INFO:workflow:Executing node trackFolderFinder in dir: /var/folders/tg/2g576vk174s3m9znw1sgjpbm0000gn/T/tmpaTzKOq/trackFolderFinder


{'/Users/srothmei/Desktop/charite/toronto/AJ_20140516_1600/mrtrix_68/masks_68/seedmask10022_1mm.nii.gz': '/Users/srothmei/Desktop/charite/toronto/AJ_20140516_1600/mrtrix_68/masks_68/targetmask1002_1mm.nii.gz', '/Users/srothmei/Desktop/charite/toronto/AJ_20140516_1600/mrtrix_68/masks_68/seedmask10021_1mm.nii.gz': '/Users/srothmei/Desktop/charite/toronto/AJ_20140516_1600/mrtrix_68/masks_68/targetmask1002_1mm.nii.gz', '/Users/srothmei/Desktop/charite/toronto/AJ_20140516_1600/mrtrix_68/masks_68/seedmask10034_1mm.nii.gz': '/Users/srothmei/Desktop/charite/toronto/AJ_20140516_1600/mrtrix_68/masks_68/targetmask1003_1mm.nii.gz', '/Users/srothmei/Desktop/charite/toronto/AJ_20140516_1600/mrtrix_68/masks_68/seedmask10035_1mm.nii.gz': '/Users/srothmei/Desktop/charite/toronto/AJ_20140516_1600/mrtrix_68/masks_68/targetmask1003_1mm.nii.gz', '/Users/srothmei/Desktop/charite/toronto/AJ_20140516_1600/mrtrix_68/masks_68/seedmask10031_1mm.nii.gz': '/Users/srothmei/Desktop/charite/toronto/AJ_20140516_1600/m

In [108]:
from nipype.utils.filemanip import split_filename
from os.path import basename

seedmask = subSet.keys()
targetmask = subSet.values()
test1 = 'Es/ist/'
test2 = '_tracks.tck'

#res = re.search("\d{4,999}", seedmask)
#r = re.compile("\d{4,999}")
#pth, fname, ext = split_filename(seedmask)
#print fname
#print pth
foo = [re.search("(\d{4,999})_.*$", basename(x)).group(1) for x in seedmask]
bar = [test1 + s + test2 for s in foo]
print foo
print bar[0]
print bar[0][:-3] + 'trk'

['10022', '10021', '10034', '10035', '10031', '10032', '10033', '10013', '10011', '10012']
Es/ist/10022_tracks.tck
Es/ist/10022_tracks.trk


In [107]:
print [basename(smsk) for smsk in seedmask]

['seedmask10022_1mm.nii.gz', 'seedmask10021_1mm.nii.gz', 'seedmask10034_1mm.nii.gz', 'seedmask10035_1mm.nii.gz', 'seedmask10031_1mm.nii.gz', 'seedmask10032_1mm.nii.gz', 'seedmask10033_1mm.nii.gz', 'seedmask10013_1mm.nii.gz', 'seedmask10011_1mm.nii.gz', 'seedmask10012_1mm.nii.gz']


In [91]:
import os, re

In [1]:
theDir = '/Users/srothmei/Desktop/charite/toronto/AJ_20140516_1600/mrtrix_68/tracks_68/'
distFiles = list()
capfiles = list()
for fileZ in os.listdir(theDir):
    if re.search("^SC_cap_row*.", fileZ):
        capfiles.append(theDir + fileZ)
    elif re.search("^SC_dist_row*.", fileZ):
        distFiles.append(theDir + fileZ)

NameError: name 'os' is not defined

In [ ]:
capfiles.sort()
distFiles.sort()
for cap_file, dist_file in zip(capfiles, distFiles):
    #print dist_file

In [40]:
import numpy as np
import os
from nipype.interfaces.base import CommandLine

theFile = '/Users/srothmei/Desktop/charite/toronto/AJ_20140516_1600/mrtrix_68/tracks_68/dlMe.txt'
tckFiles = np.loadtxt(theFile, dtype=str).tolist()

print len(tckFiles)

cmd = CommandLine(command='ls -1 ' + os.path.dirname(theFile) + '/*.tck').run()
cmd.runtime.stdout.splitlines()

INFO:interface:stdout 2015-12-18T08:03:46.015834:/Users/srothmei/Desktop/charite/toronto/AJ_20140516_1600/mrtrix_68/tracks_68/10011_tracksCN.tck
INFO:interface:stdout 2015-12-18T08:03:46.015834:/Users/srothmei/Desktop/charite/toronto/AJ_20140516_1600/mrtrix_68/tracks_68/10012_tracksCN.tck
INFO:interface:stdout 2015-12-18T08:03:46.015834:/Users/srothmei/Desktop/charite/toronto/AJ_20140516_1600/mrtrix_68/tracks_68/10013_tracksCN.tck
INFO:interface:stdout 2015-12-18T08:03:46.015834:/Users/srothmei/Desktop/charite/toronto/AJ_20140516_1600/mrtrix_68/tracks_68/10021_tracksCN.tck
INFO:interface:stdout 2015-12-18T08:03:46.015834:/Users/srothmei/Desktop/charite/toronto/AJ_20140516_1600/mrtrix_68/tracks_68/10022_tracksCN.tck
INFO:interface:stdout 2015-12-18T08:03:46.015834:/Users/srothmei/Desktop/charite/toronto/AJ_20140516_1600/mrtrix_68/tracks_68/20351_tracksCN.tck
INFO:interface:stdout 2015-12-18T08:03:46.015834:/Users/srothmei/Desktop/charite/toronto/AJ_20140516_1600/mrtrix_68/tracks_68/2035

376


[u'/Users/srothmei/Desktop/charite/toronto/AJ_20140516_1600/mrtrix_68/tracks_68/10011_tracksCN.tck',
 u'/Users/srothmei/Desktop/charite/toronto/AJ_20140516_1600/mrtrix_68/tracks_68/10012_tracksCN.tck',
 u'/Users/srothmei/Desktop/charite/toronto/AJ_20140516_1600/mrtrix_68/tracks_68/10013_tracksCN.tck',
 u'/Users/srothmei/Desktop/charite/toronto/AJ_20140516_1600/mrtrix_68/tracks_68/10021_tracksCN.tck',
 u'/Users/srothmei/Desktop/charite/toronto/AJ_20140516_1600/mrtrix_68/tracks_68/10022_tracksCN.tck',
 u'/Users/srothmei/Desktop/charite/toronto/AJ_20140516_1600/mrtrix_68/tracks_68/20351_tracksCN.tck',
 u'/Users/srothmei/Desktop/charite/toronto/AJ_20140516_1600/mrtrix_68/tracks_68/20352_tracksCN.tck',
 u'/Users/srothmei/Desktop/charite/toronto/AJ_20140516_1600/mrtrix_68/tracks_68/20353_tracksCN.tck',
 u'/Users/srothmei/Desktop/charite/toronto/AJ_20140516_1600/mrtrix_68/tracks_68/20354_tracksCN.tck']

In [43]:
import logging

logging.INFO

20

In [48]:
SC_cap_files = ['/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_1Adalberto.json', '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_2Adalberto.json', '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_3Adalberto.json', '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_4Adalberto.json', '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_5Adalberto.json', '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_6Adalberto.json', '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_7Adalberto.json', '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_8Adalberto.json', '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_9Adalberto.json', '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_10Adalberto.json', '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_11Adalberto.json', '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_12Adalberto.json', '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_13Adalberto.json', '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_14Adalberto.json', '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_15Adalberto.json', '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_16Adalberto.json', '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_17Adalberto.json', '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_18Adalberto.json', '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_19Adalberto.json', '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_20Adalberto.json', '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_21Adalberto.json', '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_22Adalberto.json', '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_23Adalberto.json', '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_24Adalberto.json', '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_25Adalberto.json', '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_26Adalberto.json', '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_27Adalberto.json', '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_28Adalberto.json', '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_29Adalberto.json', '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_30Adalberto.json', '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_31Adalberto.json', '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_32Adalberto.json', '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_33Adalberto.json', '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_34Adalberto.json', '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_35Adalberto.json', '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_36Adalberto.json', '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_37Adalberto.json', '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_38Adalberto.json', '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_39Adalberto.json', '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_40Adalberto.json', '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_41Adalberto.json', '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_42Adalberto.json', '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_43Adalberto.json', '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_44Adalberto.json', '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_45Adalberto.json', '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_46Adalberto.json', '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_47Adalberto.json', '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_48Adalberto.json', '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_49Adalberto.json', '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_50Adalberto.json', '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_51Adalberto.json', '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_52Adalberto.json', '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_53Adalberto.json', '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_54Adalberto.json', '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_55Adalberto.json', '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_56Adalberto.json', '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_57Adalberto.json', '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_58Adalberto.json', '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_59Adalberto.json', '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_60Adalberto.json', '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_61Adalberto.json', '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_62Adalberto.json', '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_63Adalberto.json', '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_64Adalberto.json', '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_65Adalberto.json', '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_66Adalberto.json', '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_67Adalberto.json', '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_68Adalberto.json']

SC_cap_files.sort()

SC_cap_files

['/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_10Adalberto.json',
 '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_11Adalberto.json',
 '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_12Adalberto.json',
 '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_13Adalberto.json',
 '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_14Adalberto.json',
 '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_15Adalberto.json',
 '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_16Adalberto.json',
 '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_17Adalberto.json',
 '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row_18Adalberto.json',
 '/home/petra/Simon/TVB-Pypeline/subjects/Adalberto/tractography/tracks/SC_cap_row